$$Exercise: \;1$$

In [32]:
!pip install -q pyomo

In [33]:
from pyomo.environ import *

In [34]:
import numpy as np

$$Q-4$$

In [35]:
coef = np.loadtxt('file1.txt', delimiter = ',')

In [36]:
print(coef.shape)
print('Number of rows=', coef.shape[0])
print('Number of columns=', coef.shape[1])

(6, 7)
Number of rows= 6
Number of columns= 7


In [37]:
print(coef)

[[  0.85   3.75  -8.75  -3.45 -22.38  20.     0.  ]
 [ -0.7   -3.9   -1.     0.     1.     0.    14.9 ]
 [  1.     0.     0.    -1.     0.    -1.     0.9 ]
 [  0.    -3.    -2.     0.     0.    -1.   -51.7 ]
 [  0.     0.     0.     4.8    9.12   7.2   75.  ]
 [ -0.8   -1.5    8.    -2.     1.     0.   -27.  ]]


$$Q-5$$

In [38]:
model = ConcreteModel()

In [39]:
# setting the number of decision variables and constraints
M = coef.shape[0] - 1 
N = coef.shape[1] - 1 

In [40]:
print(M)
print(N)

5
6


In [41]:
# coefficient of the objective function
obj_coef = coef[0,:-1]


In [42]:
# coefficients of the constraints
constr_coef = coef[1:,:-1]


In [43]:
# rhs of the constraints
constr_rhs = coef[1:,-1]


In [44]:
#Since the variables have the same upper bound and lower bound, we can simply use scalars to store them
lower_bound = 0
upper_bound = np.inf

In [45]:
# set row and column indices
row_indices = np.arange(M)
col_indices = np.arange(N)

In [46]:
print(col_indices)
print(row_indices)

[0 1 2 3 4 5]
[0 1 2 3 4]


In [47]:
# declare the decision variables in the model
model.x = Var(col_indices)

In [48]:
# create a constraint list to hold multiple constraints
model.constraints = ConstraintList()

In [49]:
for i in row_indices:
      model.constraints.add(sum(constr_coef[i][j]*model.x[j] for j in col_indices)<= constr_rhs[i])

In [50]:
# set upper and lower bounds of the variables
for j in col_indices:
  model.x[j].setlb(lower_bound)
  model.x[j].setub(upper_bound)

In [51]:
# add model objective function
model.objective = Objective(expr = summation(obj_coef, model.x), sense = maximize)

In [52]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

$$Q-6$$

In [53]:
!apt-get install -y -qq coinor-cbc

In [54]:
opt_cbc = SolverFactory('cbc')

In [55]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

    model.name="unknown";
      - termination condition: unbounded
      - message from solver: <undefined>
Solver status: warning
Solver termination condition: unbounded


$$Q-7$$

$The\; termination \;condition\; is \;unbounded,\; i.e. \;in \;this\; problem,\; we \;can \;maximize \;the\; objective \;function \;indefinitely\; without\; violating\; the\; constraints.\; The\; solver\; status\; shows \;that\; we \;cannot \;find \;an \;optimal \;solution,\; as \;the \;feasible \;region \;is\; unbounded.$

$$Q-8$$
$We\; were \;solving\; the\; problem\; in\; the\; wrong\; optimization \;sense, \;so \;the\; feasible\; region \;is\; unbounded.$

In [56]:
# reset the sense to minimize
model.objective.set_sense(minimize)
model.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

In [57]:
result = opt_cbc.solve(model)
print(result)
print('SolverStatus:', result.solver.status)
print('Termination Condition:', result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: -97.02184211
  Upper bound: -97.02184211
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 6
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.018137454986572266
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

SolverStatus: ok
Termination Condition: optimal


In [58]:
# display results
print('Objective Function:', model.objective())
print('\nDecision Variables: ')
for j in col_indices:
      print('x[',j,']= ', model.x[j].value)
print('\nConstraints:')
model.constraints.display()      

Objective Function: -97.02184239599997

Decision Variables: 
x[ 0 ]=  0.9
x[ 1 ]=  23.002456
x[ 2 ]=  0.0
x[ 3 ]=  0.0
x[ 4 ]=  8.2236842
x[ 5 ]=  0.0

Constraints:
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :         -82.1158942 :  14.9
      2 :  None :                 0.9 :   0.9
      3 :  None :          -69.007368 : -51.7
      4 :  None :   74.99999990399999 :  75.0
      5 :  None : -26.999999799999998 : -27.0


$$Q-9$$

$After\; changing\; the \;sense\; from \;maximize\;to \;minimize, \;we \;are \;now\; getting\; an\; optimal\; solution.\; The\; previous\; termination \;condition\; was\; due\; to \;wrong\; sense\; of\; the\; optimization\; problem.$ \\

$Objective \;Value=\; -97.02184$ \\

$Optimal \;solution:\; x_1 = 0.9 ;\; x_2 = 23.0024 ;\; x_3 = 0 ;\; x_4 = 0; \;x_5 = 8.2236842 ;\; x_6 = 0$

$Constraint \;2,\; 4\;and\; 5\; (last\; 2\; in\; approximate \;sense)\; are\; active\; since\; they\; attain\; exact \;(approximate) \;equality.$